In [2]:
###########
# 載入套件 #
###########
import findspark
findspark.init()
import pyspark
findspark.find()
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
############################
# 建立Spark相關語法&讀取資料 #
############################
spark = SparkSession.builder\
    .appName("tim.com")\
    .getOrCreate()
Q1_data = spark.read.csv('./2007.csv', header = True, sep = ',')
Q2_data = spark.read.csv("./2000.csv", header = True, sep = ",")
Q2_ls= ["2001", "2002", "2003", "2004", "2005"]
for i in Q2_ls:
    tmp_data = spark.read.csv("./"+i+".csv", header = True, sep = ",")
    Q2_data = Q2_data.union(tmp_data)
    print("目前正在處理",i)
Q3_data = spark.read.csv('./2008.csv', header = True, sep = ',')

目前正在處理 2001
目前正在處理 2002
目前正在處理 2003
目前正在處理 2004
目前正在處理 2005


In [3]:
######
# Q1 #
######
Q1_data.schema
Q1_data = Q1_data.withColumn("Month", col("Month").cast(IntegerType()))
Q1_data = Q1_data.withColumn("ArrDelay", col("ArrDelay").cast(IntegerType()))
Q1_data = Q1_data.withColumn("DepDelay", col("DepDelay").cast(IntegerType()))
Q1_data.groupby("Month").max("ArrDelay").sort("Month", ascending = True).show(n = 12)
Q1_data.groupby("Month").max("DepDelay").sort("Month", ascending = True).show(n = 12)
Q1_both = Q1_data.groupby("Month").max("ArrDelay","DepDelay").sort("Month", ascending = True)
Q1_both.withColumn("Maximal_Delay", greatest("max(ArrDelay)", "max(DepDelay)")).show(n=12)


+-----+-------------+
|Month|max(ArrDelay)|
+-----+-------------+
|    1|         1426|
|    2|         1359|
|    3|         1564|
|    4|         1402|
|    5|         1429|
|    6|         1351|
|    7|         1386|
|    8|         1472|
|    9|         1665|
|   10|         2598|
|   11|         1146|
|   12|         1942|
+-----+-------------+

+-----+-------------+
|Month|max(DepDelay)|
+-----+-------------+
|    1|         1406|
|    2|         1340|
|    3|         1547|
|    4|         1415|
|    5|         1416|
|    6|         1360|
|    7|         1369|
|    8|         1449|
|    9|         1689|
|   10|         2601|
|   11|         1137|
|   12|         1956|
+-----+-------------+

+-----+-------------+-------------+-------------+
|Month|max(ArrDelay)|max(DepDelay)|Maximal_Delay|
+-----+-------------+-------------+-------------+
|    1|         1426|         1406|         1426|
|    2|         1359|         1340|         1359|
|    3|         1564|         1547|         

In [4]:
######
# Q2 #
######
# Q2_data.show() # 確認資料欄位代號
# Q2_data.groupBy("_c27").count().show() # 確認需要移除的值
Q2_data = Q2_data.withColumn("Year", col("Year").cast(IntegerType()))
Q2_data_filter = Q2_data.filter(Q2_data["SecurityDelay"] != "NA")
Q2_data_filter = Q2_data_filter.filter(Q2_data_filter["SecurityDelay"] != 0)
Q2_data_filter.groupby("Year").count().sort("Year", ascending = True).show()

+----+-----+
|Year|count|
+----+-----+
|2003| 3740|
|2004| 8158|
|2005| 6627|
+----+-----+



In [4]:
######
# Q3 #
######
Q3_1_data = Q3_data.filter(Q3_data["ArrDelay"] > 0)
Q3_1_data.groupby("Dest").count().sort("count", ascending = False).show(n = 5)
Q3_1_data.groupby("Dest").count().sort("count", ascending = True).show(n = 5)
Q3_2_data = Q3_data.filter(Q3_data["DepDelay"] > 0)
Q3_2_data.groupby("Origin").count().sort("count", ascending = False).show(n = 5)
Q3_2_data.groupby("Origin").count().sort("count", ascending = True).show(n = 5)

+----+------+
|Dest| count|
+----+------+
| ATL|187243|
| ORD|151871|
| DFW|119817|
| DEN|102681|
| LAX| 93508|
+----+------+
only showing top 5 rows

+----+-----+
|Dest|count|
+----+-----+
| TUP|    2|
| PIR|    4|
| BJI|   12|
| INL|   17|
| SUX|   35|
+----+-----+
only showing top 5 rows

+------+------+
|Origin| count|
+------+------+
|   ATL|175017|
|   ORD|159427|
|   DFW|127749|
|   DEN|104414|
|   LAX| 87258|
+------+------+
only showing top 5 rows

+------+-----+
|Origin|count|
+------+-----+
|   INL|    1|
|   TUP|    1|
|   PUB|    2|
|   PIR|    3|
|   BJI|    4|
+------+-----+
only showing top 5 rows

